In [61]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import math

In [62]:
le = preprocessing.LabelEncoder()

In [63]:
data = pd.read_csv("healthcare-dataset-stroke-data.csv")

In [64]:
set(data["smoking_status"])

{'Unknown', 'formerly smoked', 'never smoked', 'smokes'}

In [65]:
data = data.drop(labels="id",axis=1)

In [66]:
data.columns

Index(['gender', 'age', 'hypertension', 'heart_disease', 'ever_married',
       'work_type', 'Residence_type', 'avg_glucose_level', 'bmi',
       'smoking_status', 'stroke'],
      dtype='object')

In [67]:
D = data.copy()

In [68]:
D.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [69]:
X = data[["gender","age","hypertension","heart_disease","ever_married","work_type","Residence_type","avg_glucose_level","bmi","smoking_status"]]
X = X.T.values
# to store the gender, ever_married, work_type, Residence_type, smoking_status
invTransform = []

aa = le.fit_transform(X[0])
invTransform.append(le.inverse_transform(aa))
D['gender']=aa

aa = le.fit_transform(X[4])
invTransform.append(le.inverse_transform(aa))
D["ever_married"] = aa

aa = le.fit_transform(X[5])
invTransform.append(le.inverse_transform(aa))
D['work_type'] = aa

aa = le.fit_transform(X[6])
invTransform.append(le.inverse_transform(aa))
D['Residence_type'] = aa

aa = le.fit_transform(X[9])
invTransform.append(le.inverse_transform(aa))
D['smoking_status'] = aa

In [70]:
D.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1,67.0,0,1,1,2,1,228.69,36.6,1,1
1,0,61.0,0,0,1,3,0,202.21,NaN,2,1
2,1,80.0,0,1,1,2,0,105.92,32.5,2,1
3,0,49.0,0,0,1,2,1,171.23,34.4,3,1
4,0,79.0,1,0,1,3,0,174.12,24.0,2,1


In [71]:
D.fillna(value = D['bmi'].median(), inplace = True)

In [72]:
d = D[D['gender'] == 'Other'].index
D.drop(d, inplace = True)

In [73]:
x = D.drop('stroke', axis = 1)
y = D['stroke']

In [74]:
from imblearn.over_sampling import RandomOverSampler

In [75]:
os = RandomOverSampler(sampling_strategy = 1)
x_ros, y_ros = os.fit_resample(x, y)
x = pd.DataFrame(x_ros)
y = pd.DataFrame(y_ros)

In [76]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")

In [77]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.75, shuffle = True, random_state = 1)
scaler = StandardScaler()
# scaler.fit(x_train)
# x_train = pd.DataFrame(scaler.fit_transform(x_train), index = x_train.index, columns = x_train.columns)
# x_test = pd.DataFrame(scaler.fit_transform(x_test), index = x_test.index, columns = x_test.columns)

In [78]:
knn = KNeighborsClassifier(n_neighbors = 2)
knn.fit(x_train, y_train)
knn_predicted = knn.predict(x_test)
knn_conf_matrix = confusion_matrix(y_test, knn_predicted)
knn_acc_score = accuracy_score(y_test, knn_predicted)

In [79]:
print("Accuracy of K-NeighborsClassifier:",knn_acc_score*100,'\n')
print(classification_report(y_test, knn_predicted))

Accuracy of K-NeighborsClassifier: 97.53187988482107 

              precision    recall  f1-score   support

           0       1.00      0.95      0.97      1200
           1       0.95      1.00      0.98      1231

    accuracy                           0.98      2431
   macro avg       0.98      0.97      0.98      2431
weighted avg       0.98      0.98      0.98      2431



RFC

In [80]:
rfc = RandomForestClassifier(100)
rfc.fit(x_train, y_train)
rfc_predicted = rfc.predict(x_test)
rfc_conf_matrix = confusion_matrix(y_test, rfc_predicted)
rfc_acc_score = accuracy_score(y_test, rfc_predicted)

In [81]:
print("Accuracy of RandomForestClassifier:",rfc_acc_score*100,'\n')
print(classification_report(y_test, rfc_predicted))

Accuracy of RandomForestClassifier: 99.09502262443439 

              precision    recall  f1-score   support

           0       1.00      0.98      0.99      1200
           1       0.98      1.00      0.99      1231

    accuracy                           0.99      2431
   macro avg       0.99      0.99      0.99      2431
weighted avg       0.99      0.99      0.99      2431

